<a href="https://colab.research.google.com/github/ss8319/Colab-Notebooks/blob/main/Language_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GPU


In [ ]:
# Confirm GPU availability
!nvidia-smi

Thu Sep  8 03:14:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#run this to clear GPU memory
#useful when we get CUDA out of memory
!pip install numba
from numba import cuda 
device = cuda.get_current_device()
device.reset()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Downloads and Setups

In [ ]:
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 2.1 MB/s 


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
! pip install datasets transformers
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 2.1 MB/s 
     |████████████████████████████████| 4.9 MB 48.3 MB/s 
     |████████████████████████████████| 212 kB 72.5 MB/s 
     |████████████████████████████████| 115 kB 72.5 MB/s 
     |████████████████████████████████| 127 kB 59.8 MB/s 
     |████████████████████████████████| 6.6 MB 65.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 2.1 MB/s 


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
# !apt install git-lfs

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

4.22.1


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

# Fine-tuning a language model

In this notebook, we'll see how to fine-tune one of the [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling tasks. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

![Widget inference representing the masked language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/masked_language_modeling.png?raw=1)

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to fine-tune a model on it.

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

#Amazon S3

In [ ]:
!pip install datasets[s3]

In [ ]:
import datasets
aws_access_key_id="E4PZ81IZBT9YXZBJP53A"
aws_secret_access_key="yVKG6BbjsBz8YQ9lk7CRPLKbO9iGAFTb6rgg4MNMzoeCIYkAoi"

s3 = datasets.filesystems.S3FileSystem(anon=True) 
s3 = datasets.filesystems.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key)  
# s3.ls('my-private-datasets/imdb/train')  

In [ ]:
#load dataset
from datasets import load_from_disk
from datasets.filesystems import S3FileSystem

s3 = S3FileSystem(anon=True)  

dataset = load_from_disk('s3aqua:aqua-ml-data/MT',fs=s3)  

print(len(dataset))

FileNotFoundError: ignored

In [ ]:
!pip install awscli
#https://github.com/ad17171717/YouTube-Tutorials/blob/main/Google%20Colab%20Tutorials/Google_Colab_Amazon_Web_Services_Command_Line_Interface_(AWS_CLI).ipynb
from IPython.display import clear_output
clear_output()

In [ ]:
#to access AWS need to confirm your security credentials, the default output format, and the default AWS region
!aws configure

In [ ]:
!aws s3 help

#SILNLP / HuggingFace Compatibility

In [ ]:
from typing import IO, Iterable, Iterator, List, Optional, Tuple, cast, Sequence
from pathlib import Path

def decode_sp(line: str) -> str:
    return line.replace(" ", "").replace("\u2581", " ").lstrip()

def decode_sp_lines(lines: Iterable[str]) -> Iterable[str]:
    return map(decode_sp, lines)

def write_corpus(corpus_path: Path, sentences: Iterable[str], append: bool = False) -> None:
    with corpus_path.open("a" if append else "w", encoding="utf-8", newline="\n") as file:
        for sentence in sentences:
            file.write(sentence + "\n")

def load_corpus(corpus_path: Path) -> Iterator[str]:
    with corpus_path.open("r", encoding="utf-8-sig") as in_file:
        for line in in_file:
            line = line.strip()
            yield line


In [ ]:
experiment_path='/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/NLLB/NLLB-HINOVBSI-GBK'

In [ ]:
write_corpus(Path(f'{experiment_path}/train.src.detok.txt'), decode_sp_lines(load_corpus(Path(f'{experiment_path}/train.src.txt'))))
write_corpus(Path(f'{experiment_path}/train.trg.detok.txt'), decode_sp_lines(load_corpus(Path(f'{experiment_path}/train.trg.txt'))))
write_corpus(Path(f'{experiment_path}/val.src.detok.txt'), decode_sp_lines(load_corpus(Path(f'{experiment_path}/val.src.txt'))))
write_corpus(Path(f'{experiment_path}/val.trg.detok.txt'), decode_sp_lines(load_corpus(Path(f'{experiment_path}/val.trg.txt'))))

#Tensorboard setup

In [ ]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter('runs/test')

#See tutorial https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html#tensorboard-setup
# HuggingFace support https://huggingface.co/docs/hub/tensorboard

#Wandb setup

In [ ]:
!pip install wandb

import wandb
wandb.login()


In [ ]:
%env WANDB_PROJECT= test_test
#%env WANDB_NOTEBOOK_NAME = Language Modelling 
#uncomment to save the code in this notebook

env: WANDB_PROJECT=test_test


# Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [ ]:
from datasets import load_dataset
#datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
import os
train_path=Path(os.path.join(experiment_path, 'train.src.detok.txt'))
val_path=Path(os.path.join(experiment_path, 'val.src.detok.txt'))
datasets = load_dataset("text", data_files={"train":train_path , "validation":val_path})

In [ ]:

##or manually change train and val path

train_path= '/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/NLLB/NLLB-HINOVBSI-GBK/gbk-GBKNT_train.txt'
val_path= '/content/gdrive/Shareddrives/Gutenberg/MT/experiments/HuggingFace/NLLB/NLLB-HINOVBSI-GBK/gbk-GBKNT_250val.txt'
datasets = load_dataset("text", data_files={"train":train_path , "validation":val_path})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-5004f13ea22a7e7b/0.0.0/21a506d1b2b34316b1e82d0bd79066905d846e5d7e619823c0dd338d6f1fa6ad. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

In [ ]:
datasets["train"][10]

{'text': 'हालांकि मतै सारै मणु संसारिक मणु रै अनुसार घमण्ड़ करदै हिन, ता मुँ भी घमण्ड़ करना।'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,अगर अहै तैस तांयै धीरज सोगी दुख सैहन्दै रैहन ता तैस सोगी राज्य भी करना अगर अहै तसेरा इन्कार करलै ता तैस भी इन्दा इन्कार करना।
1,"मणु रा पुत्र खान्दा-पिन्दा आ, अतै सो बलदै हिन कि हेरा, पेटू अतै पियक्कड़ मणु, चुंगी लैणैबाळै अतै पापी रा मित्र हा! पर तैस मणु रै कम्मा रै द्वारा पता लगदा कि सो कैतना धर्मी हा।”"
2,"प्रमात्मैं री आत्मा अक्की मणु जो ऐसा संदेश बलणै रा वरदान दिन्दा जैड़ा बुद्धि सोगी भरूरा भुन्दा, अतै सो ही आत्मा दूये मणु जो ऐह वरदान दिन्दा जैड़ा ज्ञान सोगी भरूरा भुन्दा।"
3,अबै शान्ती दिणैबाळा प्रमात्मां जिनी इन्दै प्रभु यीशु मसीह जो मुरदै थऊँ जीयाऊ जैडा भैड़ा रा महान पाहल हा अतै तिनी अपणै खूना रै द्धारा अनन्त काल रै वायदै जो स्थापित करू हा।
4,"यीशुऐ तियां सितै वलु, “जैहणै तक दूल्हा बराती सोगी हा, ता तुहै बराती थऊँ कियां व्रत करवाई सकदै?"
5,"तैहा भिड़ा मन्ज अक्क मणुऐ जोरा सितै बल्लू, “हे गुरू अऊँ तिजो सितै बिनती करदा मेरै गोबरू पुर दया-दृष्‍टि कर क्ओकि ऐह मेरा इकलौता ही पुत्र हा।"
6,"“ता सो मेसोपोटामिया परदेस थऊँ नकैई करी हारान नगर मन्ज गिच्ची बसु। तसेरै पितै री मौती थऊँ बाद प्रमात्मैं तैसिओ तैड़ी थऊँ अणीकरी ऐस देशा मन्ज बसाऊँ जैत मन्ज अबै अहै बसदै हिन,"
7,हरेक शब्द जैड़ा अहै बलदै हिन सो प्रमात्मैं री आत्मा असिओ सखाऊरा हा ना कि मणु रै ज्ञान सितै। ऐह ही प्रमात्मैं रा आत्मा इन्दी मदद करदा हा कि अहै आत्मिक गल्ला आत्मिक मणु जो सखान।
8,"ता यीशुऐ फिरी तियां सोगी बल्लू, “अऊँ तूसु सोगी सच बलदा हा, प्रमात्मैं बल्लै गाणै रा दार अऊँ हा।"
9,"हे प्यारेओ, तिन्या दर्द भरी गल्ला रै बारै मन्ज जैड़ी तुहै सहू करदै हिन तुहै हैरान मत भुन्दै क्ओकि तुहै मसीह रै हिन। पर ऐह गल्ला ठेरैतांये भूच्छु करदी हिन कि तुहै प्रमात्मैं पुर विस्वास करदै हिन कि ना।"


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or 
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`distilgpt2`](https://huggingface.co/distilgpt2) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead:

In [ ]:
model_checkpoint = 'facebook/mbart-large-cc25'
#"bigscience/bloom"
#facebook/mbart-large-cc25 #pretrained model on causal LM


To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [968,
  27620,
  6,
  89672,
  1472,
  11110,
  15525,
  41162,
  1393,
  51351,
  12686,
  9917,
  4,
  24692,
  3346,
  3045,
  838,
  183243,
  46948,
  14061,
  52848,
  69256,
  3849,
  5469,
  28233,
  6125,
  105602,
  37912,
  1682,
  150249,
  3045,
  6754,
  33575,
  998,
  22532,
  1682,
  69256,
  3849,
  3396,
  105542,
  2139,
  17251,
  66512,
  10665,
  9024,
  125,
  2,
  250004],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
# block_size = tokenizer.model_max_length
block_size = 128

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

#0:   0%|          | 0/2 [00:00<?, ?ba/s]

#1:   0%|          | 0/2 [00:00<?, ?ba/s]

#2:   0%|          | 0/2 [00:00<?, ?ba/s]

#3:   0%|          | 0/2 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'दार थू; तिनी तैसिओ गळै थऊँ पकडु अतै सो मारू जकु, अतै बल्लू, ‘जैडा कुछ मेरा कर्जा तुद दिणा सो दि दे।’ ”</s>en_XX तुद गर्भवती भूणा, अतै अक्क पुत्र जमणा। तू तसेरा नां यीशु रखैं।</s>en_XX जैहणै मसीह तम्बू मन्ज गो सो सदा तांयै अक्क बरी महापवित्र जगह मन्ज गो, तिनी अपणै सोगी बलिदान चढ़ाणै तांयै गो'

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. We will a model:

# Causal Language modeling

In [ ]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mbart-large-cc25 were not used when initializing MBartForCausalLM: ['model.encoder.layers.1.fc2.bias', 'final_logits_bias', 'model.encoder.layers.11.self_attn.out_proj.bias', 'model.encoder.layers.8.final_layer_norm.weight', 'model.encoder.embed_tokens.weight', 'model.encoder.layers.2.fc1.bias', 'model.encoder.layers.5.fc2.weight', 'model.encoder.layers.3.fc1.bias', 'model.encoder.layers.1.self_attn.k_proj.bias', 'model.encoder.layers.2.self_attn.out_proj.weight', 'model.encoder.layers.8.fc2.bias', 'model.encoder.layers.10.self_attn_layer_norm.weight', 'model.encoder.layers.5.self_attn.out_proj.bias', 'model.encoder.layers.6.self_attn_layer_norm.weight', 'model.encoder.layers.8.self_attn.q_proj.bias', 'model.encoder.layernorm_embedding.bias', 'model.encoder.layers.2.fc2.weight', 'model.encoder.layers.7.self_attn.k_proj.weight', 'model.encoder.layernorm_embedding.weight', 'model.encoder.layers.1.self_attn.v_proj.bias', 'model.encoder.laye

And some `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    report_to="wandb",  # enable logging to W&B
    #run_name="mbart_mask"  # name of the W&B run (optional)
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
)


And we can train our model:

In [ ]:
trainer.train()  # start training and logging to W&B

# post-training analysis, testing, other logged code

wandb.finish()


Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

#Hyperparameter Sweep

In [ ]:
import wandb

sweep_config = {
  "name" : "my-sweep",
  "method" : "random",
  "parameters" : {
    "epochs" : {
      "values" : [10, 20, 50]
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.1
    }
  }
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: j30v7qbp
Sweep URL: https://wandb.ai/shamus/test_test/sweeps/j30v7qbp


In [ ]:
def train():
    with wandb.init() as run:
        config = wandb.config
        model = make_model(config)
        for epoch in range(config["epochs"]):
            loss = model.fit()  # your model training code here
            wandb.log({"loss": loss, "epoch": epoch})

count = 5 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

wandb: Agent Starting Run: j29d9j8m with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.026575071621510005
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run j29d9j8m errored: NameError("name 'make_model' is not defined")
wandb: ERROR Run j29d9j8m errored: NameError("name 'make_model' is not defined")
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: aadc1z3x with config:
wandb: 	epochs: 10
wandb: 	learning_rate: 0.07552121673038732
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Run aadc1z3x errored: NameError("name 'make_model' is not defined")
wandb: ERROR Run aadc1z3x errored: NameError("name 'make_model' is not defined")
wandb: Agent Starting Run: u0hqjlme with config:
wandb: 	epochs: 50
wandb: 	learning_rate: 0.020923493017758303


wandb: Ctrl + C detected. Stopping sweep.


# Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens).

We will use the [`distilroberta-base`](https://huggingface.co/distilroberta-base) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead:

In [ ]:
model_checkpoint = "distilroberta-base"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-333e4baa6f280a66.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-23acd0930cc16da7.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-56ae8ad41a9fdf19.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-599a47a0e666ad65.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cach

And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-661796332aa2b576.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-e019d91824c225fd.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-b5875c725d0e5cb7.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-a8e3eeaa703ca023.arrow
Loading cached processed dataset at /home/sgugger/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cach

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We redefine our `TrainingArguments`:

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

Like before, the last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/bert-finetuned-wikitext2"` or `"huggingface/bert-finetuned-wikitext2"`).

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.109144,1.919501
2,1.986566,1.884295
3,1.952879,1.861102


TrainOutput(global_step=7218, training_loss=2.0377309222603213)

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 6.37


You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForMaskedLM

model = AutoModelForMaskedLM.from_pretrained("sgugger/my-awesome-model")
```